In [14]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructField, StringType, FloatType, DoubleType, DateType, StructType

import os
os.environ['PYSPARK_PYTHON'] = r'D:\anaconda\python.exe'
os.environ['JAVA_HOME'] = r'C:\BigData\Java\jdk1.8.0_321'  # Update this to your Java installation path
os.environ['SPARK_HOME'] = r'C:\BigData\spark-3.1.2-bin-hadoop3.2'

#create session in order to be capable of accessing all Spark API 
spark = SparkSession.builder.appName("Introdution to Spark DataFrame").config("spark.some.config.option", "some-value").getOrCreate()

In [32]:
purchaseSchema = StructType([
  StructField("Email", StringType(), True), 
    StructField("Address", StringType(), True), 
    StructField("Avatar", StringType(), True),
    StructField("Avg Session Length", DoubleType(), True),
    StructField("Time on App", DoubleType(), True),
    StructField("Time on Website", DoubleType(), True), 
    StructField("Length of Membership", DoubleType(), True), 
    StructField("Yearly Amount Spent", DoubleType(), True)
])
# read csv file with our defined schema into Spark DataFrame, and use "tab" delimiter 
purchaseDataframe = spark.read.csv(r"Ecommerce_Customers.csv",header=True, schema=purchaseSchema) 
#show 3 rows of our DataFrame 
purchaseDataframe.show(3)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|   Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
+--------------------+--------------------+---------+------------------+----------------

In [33]:
purchaseDataframe.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [34]:
purchaseDataframe.printSchema()


root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [35]:
num_rows = purchaseDataframe.count() 
print("number of rows: ", num_rows) 
#show our dataFrame schema 
purchaseDataframe.printSchema() 
#show statistic of the data we want
purchaseDataframe.describe('Yearly Amount Spent').show()

number of rows:  500
root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [36]:
purchaseDataframe.createOrReplaceTempView("purchaseSql") 
#select "Total" dan "Payment" column from our sql temporary  view
anotherNewDataframe = spark.sql("SELECT `Length of Membership`, `Yearly Amount Spent` FROM purchaseSql")
anotherNewDataframe.show(3)

+--------------------+-------------------+
|Length of Membership|Yearly Amount Spent|
+--------------------+-------------------+
|  4.0826206329529615|  587.9510539684005|
|    2.66403418213262|  392.2049334443264|
|   4.104543202376424| 487.54750486747207|
+--------------------+-------------------+
only showing top 3 rows



In [37]:
orderByAvatar = spark.sql("SELECT * FROM purchaseSql ORDER BY Avatar")
orderByAvatar.show(5)

+--------------------+--------------------+------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|      Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------+------------------+------------------+------------------+--------------------+-------------------+
|   flevine@gmail.com|5292 Melanie Cres...|   AliceBlue|32.204654645678524|12.480701517118197|37.680287608367976|  3.2794662600656097|  478.5842860083426|
|wagnerbrian@hotma...|50593 Wells Roads...|   AliceBlue| 32.55949313011333| 11.79779551488024|37.777365796776465|  3.1956257763719886| 468.66846559912733|
|   sfarley@jones.com|0554 Powers Curve...|AntiqueWhite| 32.39742194430904|12.055340161330117| 37.68546548721554|   3.506967573635258| 483.79652206149285|
|freemantina@canno...|870 Dennis Throug...|AntiqueWhite|32.90345433301

In [38]:
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE `Yearly Amount Spent`>200 ORDER BY `Length of Membership`")
filterAndSortWithSQL.show()

+--------------------+--------------------+------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|      Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------+------------------+------------------+------------------+--------------------+-------------------+
|sharongraves@yaho...|73098 Ronald Port...|       Wheat|32.904853667353976|12.556107616938169|37.805509432449185|  0.2699010899842742|  314.4385182951061|
|alexandra26@summe...|Unit 7032 Box 701...|        Navy| 32.83694076702139| 10.25654903128796|36.143908456341634|  0.7895199078816915| 256.67058229005585|
|fischermonica@gma...|9847 Michael Plaz...|        Navy|32.529768731474434|11.747731701242175| 36.93988205032054|  0.8015157200042076| 298.76200786180766|
|jordandavid@gmail...|61502 Parker Vist...|      Violet| 33.6666156834